# Mobile price classification with multilayer perceptron
## Tensorflow
Erik Matovič and Jakub Horvat

[Dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?select=train.csv)

Predict a price range indicating how high the price is

### 0. Imports

In [37]:
from utils import split_train_val
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
from sklearn.model_selection import train_test_split
from utils import check_null_values, print_sum_null, rescale, split_data
import numpy as np

In [38]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))


2.11.0
Num GPUs Available:  0
tf.Tensor(-1851.5519, shape=(), dtype=float32)
[]


In [39]:
wandb.login()
run=wandb.init(project="nsiete1")

### 1. Data Preprocessing

Based on [exploratory data analysis](EDA.ipynb) test set does not have target variable price_range. We split our dataset into train-dev-test.

In [40]:
# read datasets
df_train = pd.read_csv('../data/train.csv', sep=',')
df_test = df_train[-200:]
df_train = df_train[:-200]
print(df_train.shape,df_test.shape)


(1800, 21) (200, 21)


In [41]:
x_train = df_train.values[:, :-1]       #vsetko okrem price range
x_train = x_train.astype('float32')     # input data as floats
#x_train
y_train = df_train.values[:, -1]  # price range
y_train = y_train.astype('int')
print(y_train)

[1 2 2 ... 0 1 2]


In [42]:
x_test = df_test.values[:, :-1]       #vsetko okrem price range
x_test = x_test.astype('float32')     # input data as floats
#x_test
y_test = df_test.values[:, -1]  # price range
y_test = y_test.astype('int')
print(y_test)

[0 1 3 3 1 2 1 0 0 1 0 1 0 0 1 3 3 1 0 2 0 3 1 2 2 1 2 3 3 0 0 1 2 2 2 1 2
 1 0 2 3 0 0 3 0 3 0 0 3 3 0 3 0 0 0 0 3 3 2 0 0 0 0 3 0 2 3 2 3 3 2 3 1 0
 2 0 0 1 1 1 2 2 0 0 3 0 1 2 0 3 1 0 0 3 0 1 0 3 3 0 1 2 2 2 3 0 1 3 3 1 3
 0 1 0 0 2 1 0 2 0 3 1 0 2 1 2 1 3 0 2 3 1 1 1 2 1 3 2 2 2 2 3 1 0 0 0 0 2
 1 2 1 0 1 3 0 3 0 3 3 1 2 0 2 1 1 0 2 1 0 2 2 1 0 3 1 3 0 2 1 1 1 1 2 2 1
 3 1 0 1 1 0 3 0 3 3 0 2 3 0 3]


In [43]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2) #,random_state=52
print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)
class_labels = [0, 1, 2, 3]
y_cat_train = tf.keras.utils.to_categorical(y_train, num_classes=len(class_labels))
y_cat_val = tf.keras.utils.to_categorical(y_val, num_classes=len(class_labels))
y_cat_test = tf.keras.utils.to_categorical(y_test, num_classes=len(class_labels))
print(y_cat_train)

(1440, 20) (360, 20) (1440,) (360,)
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [44]:
#Define Layers and Dropout of MLP
hidden_size = 256
model = keras.Sequential([
    keras.layers.Dense(hidden_size,input_shape=(20,),activation='relu',name='l1'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(hidden_size,activation='relu',name='l2'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(hidden_size,activation='relu',name='l3'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(hidden_size,activation='relu',name='l4'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(4,activation='sigmoid',name='l5') #activation='sigmoid'
])


In [45]:
#best learning rate
lr = 0.03648030109469176
#definition of optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
#optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
#configuration of model for training
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy','mse']
              )


In [ ]:
#model training
history = model.fit(x_train,
                    y_cat_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(x_val,y_cat_val),
                    callbacks=[
                      WandbMetricsLogger(log_freq=5),
                      WandbModelCheckpoint("models")
                    ],
                    verbose=0)

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


INFO:tensorflow:Assets written to: models\assets


...

INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.0s


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.0s


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


In [ ]:
import matplotlib.pyplot as plt

#Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#prediction for test data
score=model.evaluate(x_test,y_cat_test)
predict = model.predict(x_test)
predict

In [ ]:
predicted_test = []
for row in predict:
    a = max(row)
    b=np.where(row==a)[0][0]
    predicted_test.append(b)
print(predicted_test)

In [ ]:
arr = predicted_test - y_test
a = len(arr)
b = len(y_test)
c = a-b
count = np.count_nonzero(arr == -1)
my_accuracy = (len(arr) - count)/len(arr)
print(arr,a,b,c,count,my_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
f = plt.figure(figsize=(10,6)) #plotting
f.set_size_inches(18.5, 10.5)
f.set_dpi(100)

matrix = confusion_matrix(y_test, predicted_test)
matrix_display = ConfusionMatrixDisplay(matrix, display_labels=['0', '1', '2', '3'])
matrix_display.plot(cmap='Blues')

In [ ]:
wandb.finish()
